# Testplan constructed with Pandas dataframes

## Features from the functional spec

In [1]:
block_features = [
    { 'feat' : 'pointer manager',
      'desc' : 'Design must implement a pointer manager for 2048 pointers using a freelist'},
    { 'feat' : 'Cell interface bursts',
      'desc' : 'Cell interface must allow bursts of 2 cells per cycle for up to 1000 cycles'},
    { 'feat' : 'Cell interface sustained rate',
      'desc' : 'Cell interface must support average cells rate of 1.5 cells per cycle'},
]

## Requirements from the testplan

In [2]:
block_requirements = [
    { 'req' : 'pointer manager overflow',
      'category': 'functional',
      'desc' : 'Design must never allow pointer freelist to overflow'},
    { 'req' : 'pointer manager underflow',
      'category': 'functional',
      'desc' : 'Design must never allow pointer freelist to underflow'},
    { 'req' : 'pointer manager backpressure',
      'category': 'functional',
      'desc' : 'Design must backpressure the cell pipeline when pointer resources are low'},
    { 'req' : 'pointer manager recirculate',
      'category': 'functional',
      'desc' : 'Design must demonstrate that all pointers can be used up and reused repeatedly'},    
    { 'req' : 'Cell interface bursts',
      'category': 'performance',
      'desc' : 'Cell interface must allow bursts of 2 cells per cycle for up to 1000 cycles'},
    { 'req' : 'Cell interface sustained rate',
      'category': 'performance',
      'desc' : 'Cell interface must support average cells rate of 1.5 cells per cycle'},
]

## Test coverage from verification

In [3]:
coverage = [
    {'cov' : 'ptr_mgr.ovrflw_cp1',   'grade' : 100.0},
    {'cov' : 'ptr_mgr.undflw_cp1',   'grade' : 0.0},
    {'cov' : 'ptr_mgr.bp_cp1',       'grade' : 65.0},
    {'cov' : 'ptr_mgr.all_used_cp1', 'grade' : 95.0},
    {'cov' : 'ptr_mgr.all_freed_cp1','grade' : 85.0},    
    {'cov' : 'cell_if.burst_test1',  'grade' : 33.3},
    {'cov' : 'cell_if.burst_test2',  'grade' : 100.0},    
]

# Read Data Structures into Pandas DataFrames

The data sources are all from python dictionaries. However, the data could easily be from an external file. Pandas has methods for importing from csv, json, excel and other file types.

In [4]:
import pandas as pd

## Create features dataframe

In [5]:
features_df = pd.DataFrame.from_dict(block_features)
#features_df.style.set_table_styles([dict(selector="tr",props=[('max-width', '500px')])])
features_df

,desc,feat
0,Design must implement a pointer manager for 20...,pointer manager
1,Cell interface must allow bursts of 2 cells pe...,Cell interface bursts
2,Cell interface must support average cells rate...,Cell interface sustained rate


## Create requirements dataframe

In [6]:
requirements_df = pd.DataFrame.from_dict(block_requirements)
requirements_df

,category,desc,req
0,functional,Design must never allow pointer freelist to ov...,pointer manager overflow
1,functional,Design must never allow pointer freelist to un...,pointer manager underflow
2,functional,Design must backpressure the cell pipeline whe...,pointer manager backpressure
3,functional,Design must demonstrate that all pointers can ...,pointer manager recirculate
4,performance,Cell interface must allow bursts of 2 cells pe...,Cell interface bursts
5,performance,Cell interface must support average cells rate...,Cell interface sustained rate


## Create coverage dataframe

In [7]:
coverage_df = pd.DataFrame.from_dict(coverage)
coverage_df

,cov,grade
0,ptr_mgr.ovrflw_cp1,100.0
1,ptr_mgr.undflw_cp1,0.0
2,ptr_mgr.bp_cp1,65.0
3,ptr_mgr.all_used_cp1,95.0
4,ptr_mgr.all_freed_cp1,85.0
5,cell_if.burst_test1,33.3
6,cell_if.burst_test2,100.0


# Associations

## Mapping of functional spec feature to requirements from the testplan

In [8]:
block_req_map = [
    {'feat': 'pointer manager', 'req' : 'pointer manager overflow'},
    {'feat': 'pointer manager', 'req' : 'pointer manager underflow'},
    {'feat': 'pointer manager', 'req' : 'pointer manager backpressure'},
    {'feat': 'pointer manager', 'req' : 'pointer manager recirculate'},
    {'feat': 'Cell interface bursts', 'req' : 'Cell interface bursts'}
    
]

In [9]:
f2rmap_df = pd.DataFrame.from_dict(block_req_map)
f2rmap_df

,feat,req
0,pointer manager,pointer manager overflow
1,pointer manager,pointer manager underflow
2,pointer manager,pointer manager backpressure
3,pointer manager,pointer manager recirculate
4,Cell interface bursts,Cell interface bursts


In [10]:
# how : {‘left’, ‘right’, ‘outer’, ‘inner’}, default ‘inner’
merg_df = requirements_df.merge(f2rmap_df, on='req', how='outer') #, left_index=True)

In [11]:
merg_df = merg_df.merge(features_df, suffixes=('_req','_feat'), on='feat', how='outer')
merg_df.set_index(['feat','req'])[['category','desc_req']]

category  \
feat                          req                                          
pointer manager               pointer manager overflow        functional   
                              pointer manager underflow       functional   
                              pointer manager backpressure    functional   
                              pointer manager recirculate     functional   
Cell interface bursts         Cell interface bursts          performance   
NaN                           Cell interface sustained rate  performance   
Cell interface sustained rate NaN                                    NaN   

                                                                                                      desc_req  
feat                          req                                                                               
pointer manager               pointer manager overflow       Design must never allow pointer freelist to ov...  
                              pointer manager underflow      Design must never allow pointer freelist to un...  
                              pointer manager backpressure   Design must backpressure the cell pipeline whe...  
                              pointer manager recirculate    Design must demonstrate that all pointers can ...  
Cell interface bursts         Cell interface bursts          Cell interface must allow bursts of 2 cells pe...  
NaN                           Cell interface sustained rate  Cell interface must support average cells rate...  
Cell interface sustained rate NaN                                                                          NaN

## Identify missing mappings

In [12]:
missing_req_df  =  merg_df.loc[merg_df['req'].isna()]
missing_feat_df =  merg_df.loc[merg_df['feat'].isna()]

### Features missing supporting requirements

In [13]:
display(missing_req_df)

,category,desc_req,req,feat,desc_feat
6,NaN,NaN,NaN,Cell interface sustained rate,Cell interface must support average cells rate...


### Requirements missing features

In [14]:
display(missing_feat_df)

,category,desc_req,req,feat,desc_feat
5,performance,Cell interface must support average cells rate...,Cell interface sustained rate,NaN,NaN


## Mapping of requirements from the testplan to Coverage

In [15]:
# Mapping of functional spec feature to requirements from the testplan
cov_req_map = [
    {'req' : 'pointer manager overflow',     'cov' : 'ptr_mgr.ovrflw_cp1'},
    {'req' : 'pointer manager underflow',    'cov' : 'ptr_mgr.undflw_cp1'},
    {'req' : 'pointer manager backpressure', 'cov' : 'ptr_mgr.bp_cp1'},
    {'req' : 'pointer manager recirculate',  'cov' : 'ptr_mgr.all_used_cp1'},
    {'req' : 'pointer manager recirculate',  'cov' : 'ptr_mgr.all_freed_cp1'},    
    {'req' : 'Cell interface bursts',        'cov' : 'cell_if.burst_test1'},
    {'req' : 'Cell interface bursts',        'cov' : 'cell_if.burst_test2'},
]

In [16]:
r2covmap_df = pd.DataFrame.from_dict(cov_req_map)

In [30]:
all_df = merg_df.merge(r2covmap_df, on='req', how='outer') #, left_index=True)
all_df = all_df.merge(coverage_df, on='cov', how='outer') #, left_index=True)
all_df = all_df.fillna({'grade':0})

# Report Coverage results

## Summary per Test point

In [36]:
all_df.set_index(['feat','req','cov'])[['category','desc_req','grade']].style.bar(subset=['grade'], color='#3dff3d')

## Summary per requirement

In [35]:
all_df.groupby(['feat','req']).mean().style.bar(subset=['grade'],color='#3dff3d')

In [71]:
all_df.groupby(['feat','req']).mean().style.background_gradient(subset=['grade'], cmap='RdYlGn',low=0.2,high=0.2)